https://github.com/4m4n5/fifa18-all-player-statistics/blob/master/crawler.ipynb

In [1]:
# Initial imports
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

import random
import urllib.request
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [4]:
base_url = "https://sofifa.com/players?offset="
offset = 0
columns = ['ID', 'Name', 'Photo', 'Nationality', 'Flag', 'Overall', 'Potential', 'Club', 
           'Club Logo', 'Value', 'Wage', 'Special']
data = DataFrame(columns=columns)
for offset in range(225):
    url = base_url + str(offset*80)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    table_body = soup.find('tbody')
    for row in table_body.findAll('tr'):
        td = row.findAll('td')
        picture = td[0].find('img').get('data-src')
        pid = td[0].find('img').get('id')
        nationality = td[1].find('a').get('title')
        flag_img = td[1].find('img').get('data-src')
        name = td[1].findAll('a')[1].text
        overall = td[3].text.strip()
        potential = td[4].text.strip()
        club = td[5].find('a').text
        club_logo = td[5].find('img').get('data-src')
        value = td[6].text.strip()
        wage = td[7].text.strip()
        special = td[8].text.strip()
        player_data = DataFrame([[pid, name, picture, nationality, flag_img, overall, 
                                  potential, club, club_logo, value, wage, special]])
        player_data.columns = columns
        data = data.append(player_data, ignore_index=True)
    offset+=1
    data.to_csv('full_player_data.csv', encoding='utf-8')
    if (offset % 20 == 0):
        print(offset)


20
40
60
80
100
120
140
160
180
200
220


In [6]:
data.to_csv('basicplayerdata.csv', encoding='utf-8')

In [7]:
player_data_url = 'https://sofifa.com/player/'
for index, row in data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'column col-4 mb-20'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    master_data = DataFrame(columns=skill_names)
    break

In [8]:
master_data.columns

Index([], dtype='object')

In [10]:
player_data_url = 'https://sofifa.com/player/'
r = 0
for index, row in data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'column col-4 mb-20'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    attr_data = DataFrame(columns=skill_names)
    for key in skill_map.keys():
        attr_data.loc[r,key] = skill_map[key]
    r = r + 1
    master_data = pd.concat([master_data, attr_data])
    if r % 1000 == 0:
        print(r)
        master_data.to_csv('PlayerAttributeData.csv', encoding='utf-8')


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [11]:
master_data.head()

,ID
0,236988
1,255253
2,239301
3,256958
4,236403


In [12]:
full_data = pd.merge(data, master_data, left_index=True, right_index=True)

In [13]:
master_data.to_csv('PlayerAttributeData.csv', encoding='utf-8')

In [14]:
full_data.to_csv('Dataset.csv', encoding='utf-8')

In [17]:
# full_data.drop('Unnamed: 0', 1,  inplace=True)

In [18]:
full_data.drop('ID_x', 1,  inplace=True)

In [19]:
full_data['ID_y'][:5]

0    236988
0    236988
1    255253
1    255253
2    239301
Name: ID_y, dtype: object

In [20]:
f = full_data.rename(index=str, columns={"ID_y": "ID"})

In [21]:
f['ID'][:5]

0    236988
0    236988
1    255253
1    255253
2    239301
Name: ID, dtype: object

In [22]:
f.to_csv('Dataset.csv', encoding='utf-8')